In [2]:
import geopandas as gpd
import pandas as pd
import os

import sys
sys.path.append('/Users/ivanesipov/Desktop/Учеба/Курсовая_3/Coursework_3/src/utils')
import parsutils as pu

In [13]:
# загружаем список городов
cities_path = "/Users/ivanesipov/Desktop/Учеба/Курсовая_3/Coursework_3/references/cities_list_OSM.txt"
L1 = open(cities_path).readlines()
cities = []
for city in L1:
    cities.append(city.replace('\n',''))

In [ ]:
# разбиваем формы городов на полигоны, после чего разбиваем их на гексагоны
poly_cities = pu.split_multipolygons(cities).reset_index().drop(columns = 'index')
hexagons = pu.hexagons_from_cities(poly_cities, hexagon_size = 5)

In [ ]:
# объединяем объекты из папки с городами в один датафрейм и сохраним в data/interim/
path = '/Users/ivanesipov/Desktop/Учеба/Курсовая_3/Coursework_3/data/raw/finals'
pu.concat_cities_objects(path)

In [64]:
# считываем файл из предыдущей ячейки
f = '/Users/ivanesipov/Desktop/Учеба/Курсовая_3/Coursework_3/data/interim/final_concatenation.csv'
df = pd.read_csv(f)
df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])

In [65]:
# обрабатываем данные из датафрейма со всеми объектами и пересекаем их с полигонами
gdf = gpd.GeoDataFrame(df, geometry= 'geometry').iloc[:,3:].drop(['lat', 'lon'], axis = 1)
gdf_joined = gpd.sjoin(hexagons, gdf, 'inner')
gdf_joined.drop(['index_right'], axis = 1, inplace=True)
gdf_joined.reset_index(inplace = True)
gdf_joined.rename(columns = {'index':'hex_index'}, inplace = True)

In [66]:
# посчитаем объекты в полигонах и сохраним в словаре
Final = pu.count_objects(gdf_joined)

100%|██████████| 600575/600575 [6:49:39<00:00, 24.43it/s]    


In [69]:
# сохраняем данные
os.chdir('/Users/ivanesipov/Desktop/Учеба/Курсовая_3/Coursework_3/data/processed')
Final.to_csv('Final_hexes_dict_counter.csv')

In [16]:
# для демонстрации (
demo = pd.read_csv('/Users/ivanesipov/Desktop/Учеба/Курсовая_3/Coursework_3/data/processed/Final_hexes_dict_counter.csv')
demo.rename(columns = {'0': 'hexagon_id'}, inplace = True)
demo.drop('Unnamed: 0', axis = 1, inplace = True)
demo.sample(n=10)

,hexagon_id,city,geometry,counter_dict
112182,42530,Saint-Petersburg,"POLYGON ((30.4574778373787 60.00520816314822, ...",Counter({'waterway_ditch': 1})
110852,42359,Saint-Petersburg,"POLYGON ((30.20764017916242 59.81909104808532,...",Counter({'waterway_ditch': 5})
582838,601607,Moscow,"POLYGON ((37.73764602777381 55.66526869669656,...","Counter({'highway_service': 4, 'landuse_constr..."
295029,294207,Yekaterinburg,"POLYGON ((60.32577759826023 56.9089979894114, ...","Counter({'natural_wood': 4, 'highway_track': 2..."
298722,244830,Yekaterinburg,"POLYGON ((60.65142653543182 56.8775982196138, ...","Counter({'railway_rail': 14, 'building_industr..."
311574,260950,Yekaterinburg,"POLYGON ((60.61475238325298 56.9400826374019, ...","Counter({'highway_trunk': 4, 'highway_trunk_li..."
281750,230236,Yekaterinburg,"POLYGON ((60.92030337876403 56.81051617973647,...","Counter({'natural_wood': 2, 'highway_track': 1})"
6807,28494,Saint-Petersburg,"POLYGON ((30.3614427028565 59.95344935126755, ...","Counter({'waterway_river': 2, 'water_river': 1..."
554717,561238,Уфа,"POLYGON ((56.06703336868505 54.68377921292708,...",Counter({'natural_wood': 3})
511763,536364,Ростов-на-Дону,"POLYGON ((39.74174181139296 47.33229872480777,...","Counter({'highway_path': 4, 'natural_wood': 1})"
